In [47]:
%load_ext autoreload

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [48]:
%autoreload 2
%aimport AD_predictor_tools
%aimport AD_comparison_tools
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
import AD_predictor_tools
import protfasta
import warnings
warnings.filterwarnings('ignore')
import sys
import io


In [49]:
# Soto AD list
Soto = pd.read_csv("../data/SotoEtAl_ADs.csv")
Soto

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,Unnamed: 0.1.1.1,Unnamed: 0.1.1.1.1,Effector domain ID,GeneName,TF Family,Domain type,uniprotID,...,ENSEMBL gene ID,ENSEMBL protein ID,Assay,Reference (PMID),Notes,N or S?,"Activity (H, M or L)","Confidence (H, M or L)",Start,End
0,0,0,0,0,2,Effector 0003,AHCTF1,AT hook,AD,Q8WYP5,...,ENSG00000153207,ENSP00000497061,Gal4 DB fusion and luciferase assay,11952839,NaN,S,H,H,1445,1698
1,1,1,1,1,4,Effector 0005,AHR,bHLH,AD,P35869,...,ENSG00000106546,ENSP00000242057,LexA DB and Gal4 DB fusions and b-galactosidas...,8794892,NaN,N and S,H,H,545,848
2,2,2,2,2,5,Effector 0006,AIRE,SAND,AD,O43918,...,ENSG00000160224,ENSP00000291582,Luciferase assay,11274163,NaN,S,M,H,292,545
3,3,3,3,3,6,Effector 0007,AKAP8L,C2H2 ZF,AD,Q9ULX6,...,ENSG00000011243,ENSP00000380557,CAT assay,11402034,NaN,N and S,M,H,1,384
4,4,4,4,4,7,Effector 0008,ALX1,Homeodomain,AD,Q15699,...,ENSG00000180318,ENSP00000315417,Gal4 DB fusion and luciferase assay,12390248,based on rat,S,H,H,237,281
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
515,515,515,515,515,869,Effector 0870,ZNF326,C2H2 ZF,AD,Q5BKZ1,...,ENSG00000162664,ENSP00000340796,Luciferase assay,30401956,NaN,N,M,M,1,124
516,516,516,516,516,884,Effector 0885,ZNF426,C2H2 ZF,AD,Q9BUY5,...,ENSG00000130818,ENSP00000253115,Gal4 DB fusion and luciferase assay,17409159,NaN,S,H,H,99,213
517,517,517,517,517,903,Effector 0904,ZNF641,C2H2 ZF,AD,Q96N77,...,ENSG00000167528,ENSP00000437832,Gal4 DB fusion and luciferase assay,16343441,NaN,S,M,H,171,262
518,518,518,518,518,922,Effector 0923,ZXDA,C2H2 ZF,AD,P98168,...,ENSG00000198205,ENSP00000351530,Gal4 DB fusion and luciferase assay,17493635,NaN,N,H,M,572,698


In [50]:
Soto_uniprotADs = Soto[["uniprotID"]]
Soto_uniprotADs.to_csv("../data/SotoUniprotADs.csv")
Soto_uniprotADs

,uniprotID
0,Q8WYP5
1,P35869
2,O43918
3,Q9ULX6
4,Q15699
...,...
515,Q5BKZ1
516,Q9BUY5
517,Q96N77
518,P98168


1. Used uniprot retreive/mapping tool
2. Uploaded csv file above with the uniprotADs, downloaded fasta with the sequences

In [51]:
Soto_Sequences = AD_predictor_tools.makeFullLengthProteinDF("../data/Soto_Sequences")
Soto_Sequences

There are 375 proteins


,GeneName,AAseq
0,sp|Q8WYP5|ELYS_HUMAN Protein ELYS OS=Homo sapi...,MRDLRAQVTSGLLPFPEVTLQALGEDEITLESVLRGKFAAGKNGLA...
1,sp|P35869|AHR_HUMAN Aryl hydrocarbon receptor ...,MNSSSANITYASRKRRKPVQKTVKPIPAEGIKSNPSKRHRDRLNTE...
2,sp|O43918|AIRE_HUMAN Autoimmune regulator OS=H...,MATDAALRRLLRLHRTEIAVAVDSAFPLLHALADHDVVPEDKFQET...
3,sp|Q9ULX6|AKP8L_HUMAN A-kinase anchor protein ...,MSYTGFVQGSETTLQSTYSDTSAQPTCDYGYGTWNSGTNRGYEGYG...
4,sp|Q15699|ALX1_HUMAN ALX homeobox protein 1 OS...,MEFLSEKFALKSPPSKNSDFYMGAGGPLEHVMETLDNESFYSKASA...
...,...,...
370,sp|Q5BKZ1|ZN326_HUMAN DBIRD complex subunit ZN...,MDFEDDYTHSACRNTYQGFNGMDRDYGPGSYGGMDRDYGHGSYGGQ...
371,sp|Q9BUY5|ZN426_HUMAN Zinc finger protein 426 ...,MAAADLSHGHYLSGDPVCLHEEKTPAGRIVADCLTDCYQDSVTFDD...
372,sp|Q96N77|ZN641_HUMAN Zinc finger protein 641 ...,MQAEDRSQFGSAAEMLSEQTAALGTGWESMNVQLDGAEPQVERGSQ...
373,sp|P98168|ZXDA_HUMAN Zinc finger X-linked prot...,MEIPKLLPARGTLQGGGGGGIPAGGGRVHRGPDSPAGQVPTRRLLL...


In [52]:
Soto_Sequences["uniprotID"] = [a.split("|")[1] for a in Soto_Sequences["GeneName"]]
Soto_Sequences

,GeneName,AAseq,uniprotID
0,sp|Q8WYP5|ELYS_HUMAN Protein ELYS OS=Homo sapi...,MRDLRAQVTSGLLPFPEVTLQALGEDEITLESVLRGKFAAGKNGLA...,Q8WYP5
1,sp|P35869|AHR_HUMAN Aryl hydrocarbon receptor ...,MNSSSANITYASRKRRKPVQKTVKPIPAEGIKSNPSKRHRDRLNTE...,P35869
2,sp|O43918|AIRE_HUMAN Autoimmune regulator OS=H...,MATDAALRRLLRLHRTEIAVAVDSAFPLLHALADHDVVPEDKFQET...,O43918
3,sp|Q9ULX6|AKP8L_HUMAN A-kinase anchor protein ...,MSYTGFVQGSETTLQSTYSDTSAQPTCDYGYGTWNSGTNRGYEGYG...,Q9ULX6
4,sp|Q15699|ALX1_HUMAN ALX homeobox protein 1 OS...,MEFLSEKFALKSPPSKNSDFYMGAGGPLEHVMETLDNESFYSKASA...,Q15699
...,...,...,...
370,sp|Q5BKZ1|ZN326_HUMAN DBIRD complex subunit ZN...,MDFEDDYTHSACRNTYQGFNGMDRDYGPGSYGGMDRDYGHGSYGGQ...,Q5BKZ1
371,sp|Q9BUY5|ZN426_HUMAN Zinc finger protein 426 ...,MAAADLSHGHYLSGDPVCLHEEKTPAGRIVADCLTDCYQDSVTFDD...,Q9BUY5
372,sp|Q96N77|ZN641_HUMAN Zinc finger protein 641 ...,MQAEDRSQFGSAAEMLSEQTAALGTGWESMNVQLDGAEPQVERGSQ...,Q96N77
373,sp|P98168|ZXDA_HUMAN Zinc finger X-linked prot...,MEIPKLLPARGTLQGGGGGGIPAGGGRVHRGPDSPAGQVPTRRLLL...,P98168


In [53]:
full_length_seqs = Soto.merge(Soto_Sequences, left_on='uniprotID', right_on='uniprotID')
AAseq = [seq[start: end+1] for seq, start, end in zip(full_length_seqs["AAseq"], full_length_seqs["Start"], full_length_seqs["End"])]
full_length_seqs


,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,Unnamed: 0.1.1.1,Unnamed: 0.1.1.1.1,Effector domain ID,GeneName_x,TF Family,Domain type,uniprotID,...,Assay,Reference (PMID),Notes,N or S?,"Activity (H, M or L)","Confidence (H, M or L)",Start,End,GeneName_y,AAseq
0,0,0,0,0,2,Effector 0003,AHCTF1,AT hook,AD,Q8WYP5,...,Gal4 DB fusion and luciferase assay,11952839,NaN,S,H,H,1445,1698,sp|Q8WYP5|ELYS_HUMAN Protein ELYS OS=Homo sapi...,MRDLRAQVTSGLLPFPEVTLQALGEDEITLESVLRGKFAAGKNGLA...
1,1,1,1,1,4,Effector 0005,AHR,bHLH,AD,P35869,...,LexA DB and Gal4 DB fusions and b-galactosidas...,8794892,NaN,N and S,H,H,545,848,sp|P35869|AHR_HUMAN Aryl hydrocarbon receptor ...,MNSSSANITYASRKRRKPVQKTVKPIPAEGIKSNPSKRHRDRLNTE...
2,2,2,2,2,5,Effector 0006,AIRE,SAND,AD,O43918,...,Luciferase assay,11274163,NaN,S,M,H,292,545,sp|O43918|AIRE_HUMAN Autoimmune regulator OS=H...,MATDAALRRLLRLHRTEIAVAVDSAFPLLHALADHDVVPEDKFQET...
3,3,3,3,3,6,Effector 0007,AKAP8L,C2H2 ZF,AD,Q9ULX6,...,CAT assay,11402034,NaN,N and S,M,H,1,384,sp|Q9ULX6|AKP8L_HUMAN A-kinase anchor protein ...,MSYTGFVQGSETTLQSTYSDTSAQPTCDYGYGTWNSGTNRGYEGYG...
4,4,4,4,4,7,Effector 0008,ALX1,Homeodomain,AD,Q15699,...,Gal4 DB fusion and luciferase assay,12390248,based on rat,S,H,H,237,281,sp|Q15699|ALX1_HUMAN ALX homeobox protein 1 OS...,MEFLSEKFALKSPPSKNSDFYMGAGGPLEHVMETLDNESFYSKASA...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
504,515,515,515,515,869,Effector 0870,ZNF326,C2H2 ZF,AD,Q5BKZ1,...,Luciferase assay,30401956,NaN,N,M,M,1,124,sp|Q5BKZ1|ZN326_HUMAN DBIRD complex subunit ZN...,MDFEDDYTHSACRNTYQGFNGMDRDYGPGSYGGMDRDYGHGSYGGQ...
505,516,516,516,516,884,Effector 0885,ZNF426,C2H2 ZF,AD,Q9BUY5,...,Gal4 DB fusion and luciferase assay,17409159,NaN,S,H,H,99,213,sp|Q9BUY5|ZN426_HUMAN Zinc finger protein 426 ...,MAAADLSHGHYLSGDPVCLHEEKTPAGRIVADCLTDCYQDSVTFDD...
506,517,517,517,517,903,Effector 0904,ZNF641,C2H2 ZF,AD,Q96N77,...,Gal4 DB fusion and luciferase assay,16343441,NaN,S,M,H,171,262,sp|Q96N77|ZN641_HUMAN Zinc finger protein 641 ...,MQAEDRSQFGSAAEMLSEQTAALGTGWESMNVQLDGAEPQVERGSQ...
507,518,518,518,518,922,Effector 0923,ZXDA,C2H2 ZF,AD,P98168,...,Gal4 DB fusion and luciferase assay,17493635,NaN,N,H,M,572,698,sp|P98168|ZXDA_HUMAN Zinc finger X-linked prot...,MEIPKLLPARGTLQGGGGGGIPAGGGRVHRGPDSPAGQVPTRRLLL...


In [54]:
#Ending at position n includes position n
Soto_withUniprotADs = full_length_seqs
Soto_withUniprotADs["AAseq"] = AAseq
Soto_withUniprotADs = Soto_withUniprotADs.rename(columns = {"AAseq":"ProteinRegionSeq"})
Soto_withUniprotADs.to_csv("../data/Soto_withUniprotADs.csv")
Soto_withUniprotADs

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,Unnamed: 0.1.1.1,Unnamed: 0.1.1.1.1,Effector domain ID,GeneName_x,TF Family,Domain type,uniprotID,...,Assay,Reference (PMID),Notes,N or S?,"Activity (H, M or L)","Confidence (H, M or L)",Start,End,GeneName_y,ProteinRegionSeq
0,0,0,0,0,2,Effector 0003,AHCTF1,AT hook,AD,Q8WYP5,...,Gal4 DB fusion and luciferase assay,11952839,NaN,S,H,H,1445,1698,sp|Q8WYP5|ELYS_HUMAN Protein ELYS OS=Homo sapi...,RANDNKSMADVLGDGGNSSLTISEGPIVSERRLNQEVALNLKEDHE...
1,1,1,1,1,4,Effector 0005,AHR,bHLH,AD,P35869,...,LexA DB and Gal4 DB fusions and b-galactosidas...,8794892,NaN,N and S,H,H,545,848,sp|P35869|AHR_HUMAN Aryl hydrocarbon receptor ...,LGIDFEDIRHMQNEKFFRNDFSGEVDFRDIDLTDEILTYVQDSLSK...
2,2,2,2,2,5,Effector 0006,AIRE,SAND,AD,O43918,...,Luciferase assay,11274163,NaN,S,M,H,292,545,sp|O43918|AIRE_HUMAN Autoimmune regulator OS=H...,QKNEDECAVCRDGGELICCDGCPRAFHLACLSPPLREIPSGTWRCS...
3,3,3,3,3,6,Effector 0007,AKAP8L,C2H2 ZF,AD,Q9ULX6,...,CAT assay,11402034,NaN,N and S,M,H,1,384,sp|Q9ULX6|AKP8L_HUMAN A-kinase anchor protein ...,SYTGFVQGSETTLQSTYSDTSAQPTCDYGYGTWNSGTNRGYEGYGY...
4,4,4,4,4,7,Effector 0008,ALX1,Homeodomain,AD,Q15699,...,Gal4 DB fusion and luciferase assay,12390248,based on rat,S,H,H,237,281,sp|Q15699|ALX1_HUMAN ALX homeobox protein 1 OS...,SCMLPRDTSSCMTPYSHSPRTDSSYTGFSNHQNQFSHVPLNNFFT
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
504,515,515,515,515,869,Effector 0870,ZNF326,C2H2 ZF,AD,Q5BKZ1,...,Luciferase assay,30401956,NaN,N,M,M,1,124,sp|Q5BKZ1|ZN326_HUMAN DBIRD complex subunit ZN...,DFEDDYTHSACRNTYQGFNGMDRDYGPGSYGGMDRDYGHGSYGGQR...
505,516,516,516,516,884,Effector 0885,ZNF426,C2H2 ZF,AD,Q9BUY5,...,Gal4 DB fusion and luciferase assay,17409159,NaN,S,H,H,99,213,sp|Q9BUY5|ZN426_HUMAN Zinc finger protein 426 ...,RTVQGGVLQGWEMRLETQWSILQQDFLRGQTSIGIQLEGKHNGREL...
506,517,517,517,517,903,Effector 0904,ZNF641,C2H2 ZF,AD,Q96N77,...,Gal4 DB fusion and luciferase assay,16343441,NaN,S,M,H,171,262,sp|Q96N77|ZN641_HUMAN Zinc finger protein 641 ...,DPQDLEERDILRVTYTGDGSEHEGDTPELEAEPPRMLSSVSEDTVL...
507,518,518,518,518,922,Effector 0923,ZXDA,C2H2 ZF,AD,P98168,...,Gal4 DB fusion and luciferase assay,17493635,NaN,N,H,M,572,698,sp|P98168|ZXDA_HUMAN Zinc finger X-linked prot...,DLLAQLEAANSLTPSSELTSQRQNDLSDAEIVSLFSDVPDSTSAAL...
